In [3]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import RandomState

import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import time
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential

import glob
import re
from tqdm import tqdm
import statistics
import pathlib

from sklearn.preprocessing import MinMaxScaler

from keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM
from keras.models import Sequential
import tensorflow as tf
import warnings
from sklearn.model_selection import train_test_split
from torch.autograd import Variable

from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

/Users/sue/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [132]:
log =  pd.read_csv('log_mini.csv', low_memory=False)
track_f =  pd.read_csv('tf_mini.csv', low_memory=False)

# all track featrue data 
# track_f1= pd.read_csv('tf_000000000000.csv', low_memory=False)
# track_f2 = pd.read_csv('tf_000000000001.csv', low_memory=False)
# tf_all = pd.concat([track_f1,track_f2])

In [133]:
def trackfeatures_preprocessing(track_f):
    # build track_id dictionary 
    track_id_dict = pd.unique(track_f.track_id)
    track_id_dict = pd.Series(np.arange(len(track_id_dict)), track_id_dict).to_dict()
    
    # convert major/minor  to boolean
    track_f['mode'] = track_f['mode'].apply(lambda x: True if 'major' in x else False)
    
    # standardize 
    scaler = StandardScaler()
    track_f = scaler.fit_transform(track_f.iloc[:,1:])
    
    return track_f, track_id_dict

In [134]:
track_f, track_id_dict = trackfeatures_preprocessing(track_f)

/Users/sue/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/sue/anaconda3/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [135]:
def session_preprocessing(log):
    
    # build session id dictionary 
    session_id_dict = pd.unique(log.session_id)
    session_id_dict = pd.Series(np.arange(len(session_id_dict)), session_id_dict).to_dict()
    
    # replace track_id in session data 
    log.loc[:, "track_id_clean"] = np.vectorize(track_id_dict.get)(log.track_id_clean)
    
    #split date 
#     log['year'] =  pd.to_datetime(log['date'], format='%Y-%m-%d').dt.year 
#     log['month'] = pd.to_datetime(log['date'], format='%Y-%m-%d').dt.month
#     log['day'] = pd.to_datetime(log['date'], format='%Y-%m-%d').dt.day
#     log = log.drop(['date'],axis = 1)

    # Min-max scaler for numerical data
#     scaler = MinMaxScaler()
#     numer_columns = ['hist_user_behavior_n_seekfwd','hist_user_behavior_n_seekback']
#     log[numer_columns] = pd.DataFrame(scaler.fit_transform(log[numer_columns]))
    
    # One-hot encoding for categorical data 
#     categ_columns = ['skip_1','skip_2','skip_3','not_skipped','context_switch','no_pause_before_play','short_pause_before_play','long_pause_before_play',
#                'hist_user_behavior_is_shuffle','hour_of_day','day','month','year', 'premium','context_type','hist_user_behavior_reason_start',
#                'hist_user_behavior_reason_end']
#     log = pd.get_dummies(log, columns = categ_columns)
    log['skip_2'] = log['skip_2']*1
    return log,session_id_dict

In [136]:
log,session_id_dict = session_preprocessing(log)

In [137]:
track_f_tensor = torch.FloatTensor(track_f)   

In [138]:
def Input_prepare(data):
    position = np.array(data['session_position'])
    length = np.array(data['session_length'])
    total_session = []
    start = 0
    for i in range(len(position)):
        if position[i] == length[i]:
            end = i
            total_session.append(torch.LongTensor(data['track_id_clean'][start:end].values))
            start = i+1
    Train_X = total_session[0:8000]
    Test_X = total_session[8000:]
    return Train_X,Test_X

In [139]:
Train_X,Test_X = Input_prepare(log)

In [140]:
def label_prepare(data):
    # split session 
    session_split_idx = np.empty(shape=(0,), dtype=np.int32)
    index_start_from = 0
    session_split_idx = np.hstack((session_split_idx, np.insert(np.cumsum(np.unique(data.session_id.values, return_counts=True)[1])[:-1], 0, 0) + index_start_from))   
    session_split_idx = session_split_idx[1:]
    
    Y = []
    for i in session_split_idx:
        Y.append(data['skip_2'][i-1] )
    Train_Y = Y[0:8000]
    Test_Y = Y[8000:]
    return Train_Y,Test_Y

In [141]:
Train_Y, Test_Y = label_prepare(log)

In [142]:
def candidate_prepare(data):
    # split session 
    session_split_idx = np.empty(shape=(0,), dtype=np.int32)
    index_start_from = 0
    session_split_idx = np.hstack((session_split_idx, np.insert(np.cumsum(np.unique(data.session_id.values, return_counts=True)[1])[:-1], 0, 0) + index_start_from))   
    session_split_idx = session_split_idx[1:]
    
    candidate_session = []
    for i in session_split_idx:
        candidate_session.append((data['track_id_clean'][i-1]))
    candidate_session = torch.LongTensor(candidate_session) 
    Train_X_candi = candidate_session[0:8000]
    Test_X_candi = candidate_session[8000:]
    return Train_X_candi,Test_X_candi

In [143]:
Train_X_candi,Test_X_candi = candidate_prepare(log)

In [147]:
#log_1 = log[0:10007]

In [149]:
#position_1 =position [0:10007]

In [150]:
#session_split_idx_1 = session_split_idx[0:591]

In [152]:
# combine track feature and get new track embedding  dimension = 58 
# for i in range(len(position_1)):
#     for j in session_split_idx_1:
#         if position_1[i] < j:
#             tf[log_1['track_id_clean'][i]] = np.append(track_f[log_1['track_id_clean'][i]], track_f[log_1['track_id_clean'][j-1]])

In [ ]:
# np.savetxt('track_feature_log',(tf))

In [ ]:
# tf =np.loadtxt('track_feature_log')

In [144]:
class LSTMClassifier(nn.Module):
    def __init__(self,weight_matrix_of_track, embedding_dim, hidden_dim, num_layers, num_directions ,batch_size):
        super(LSTMClassifier, self).__init__()
        self.hidden_dim=hidden_dim
        self.num_layers = num_layers
        self.num_directions = num_directions 
        self.batch_size=batch_size
        self.embedding_dim = embedding_dim  
        self.embedding_track= nn.Embedding.from_pretrained(weight_matrix_of_track)     # track embedding layer
        self.lstm=nn.LSTM(self.embedding_dim,self.hidden_dim,batch_first=True)
        self.ln1 = nn.Linear(hidden_dim, 128)
        self.ln2 = nn.Linear(128,1)
        
#     def init_hidden(self):
#         return(Variable(torch.zeros(1,self.batch_size,self.hidden_dim)),Variable(torch.zeros(1,self.batch_size,self.hidden_dim)))
    def forward(self,session,candidate_session): 
        embeds_session = self.embedding_track(session)    # session embedding 
        embeds_candidate_session = self.embedding_track(candidate_session)
        
        # empand to current embeds_session size 
        embeds_candidate_session = embeds_candidate_session.expand( len(session), 29)
        # merge two embeddings into one 
        merged = torch.cat([embeds_session,embeds_candidate_session],1)
        merged_session = merged.view(self.batch_size,-1,self.embedding_dim)
        h0 = Variable(torch.zeros(self.batch_size,self.num_layers*self. num_directions, self.hidden_dim))
        c0 = Variable(torch.zeros(self.batch_size,self.num_layers*self. num_directions, self.hidden_dim))
        
        lstm_out, _ = self.lstm( merged_session, (h0, c0))
        
        out = F.sigmoid(self.ln1(lstm_out[:,-1,:]))
        output = F.sigmoid(self.ln2(out))
        return output

In [145]:
# define accuracy
def accuracy(test_session, Test_X_candi, test_label,model):
    total = len(test_label)
    correct=0
    for session,candidate_session, labels in zip(test_session, Test_X_candi,  test_label):  
        labels = torch.LongTensor([test_label])
        _, predictions = torch.max(model(session,candidate_session), dim = 1)             
        correct += (predictions == labels).sum()
        return float(correct)/total  

In [146]:
model = LSTMClassifier(weight_matrix_of_track =track_f_tensor , embedding_dim = 58 , hidden_dim = 256 , num_layers = 1,
                     num_directions =1 , batch_size = 1)
criteria =nn.BCELoss()
optimizer=optim.Adam(model.parameters(),lr=0.01)
epoch_accuracies = []
best_accuracy=0
No_epochs = 2

for epoch in range(No_epochs):
    running_loss = 0
    i=0
    for session,candidate_session, labels in zip(Train_X,Train_X_candi,Train_Y):
        i=i+1
        labels = torch.FloatTensor([labels])
        optimizer.zero_grad()
        outputs = model(session,candidate_session)     
        # labels=labels.unsqueeze(dim=0)    
        outputs = torch.squeeze(outputs)
        loss = criteria(outputs,labels)
        running_loss += loss.data[0] 
        loss.backward()
        optimizer.step()
        if (i+1)%1000==0:
            print ('Epoch: [%d/%d], Step: [%d/%d], Loss: %.4f, Train_Acc: %.4f'
                     % (epoch+1, 10, i+1, len(Train_X), 
                        loss.data, accuracy(Train_X,Train_X_candi,Train_Y,model) ))
    epoch_accuracies.append(accuracy(Test_X,Test_X_candi, Test_Y,model))

#         if epoch_accuracies[-1] > best_accuracy:
#     best_accuracy = epoch_accuracies[-1]
#     best_epoch = epoch
#     if(epoch>=5 and epoch_accuracies[-1]<=epoch_accuracies[-5]):
#     break        
#   # TODO: implement early stopping here      
#   print(best_accuracy, best_epoch)

# Q batch size ??

/Users/sue/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/Users/sue/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1594: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))
/Users/sue/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Epoch: [1/10], Step: [1000/8000], Loss: 0.9987, Train_Acc: 0.5974
Epoch: [1/10], Step: [2000/8000], Loss: 0.6399, Train_Acc: 0.5974
Epoch: [1/10], Step: [3000/8000], Loss: 0.3831, Train_Acc: 0.5974
Epoch: [1/10], Step: [4000/8000], Loss: 0.3820, Train_Acc: 0.5974
Epoch: [1/10], Step: [5000/8000], Loss: 0.5796, Train_Acc: 0.5974
Epoch: [1/10], Step: [6000/8000], Loss: 0.5629, Train_Acc: 0.5974
Epoch: [1/10], Step: [7000/8000], Loss: 0.5128, Train_Acc: 0.5974
Epoch: [1/10], Step: [8000/8000], Loss: 0.7330, Train_Acc: 0.5974
Epoch: [2/10], Step: [1000/8000], Loss: 0.5105, Train_Acc: 0.5974
Epoch: [2/10], Step: [2000/8000], Loss: 0.5564, Train_Acc: 0.5974
Epoch: [2/10], Step: [3000/8000], Loss: 0.4448, Train_Acc: 0.5974
Epoch: [2/10], Step: [4000/8000], Loss: 0.3940, Train_Acc: 0.5974
Epoch: [2/10], Step: [5000/8000], Loss: 0.5310, Train_Acc: 0.5974
Epoch: [2/10], Step: [6000/8000], Loss: 0.5139, Train_Acc: 0.5974
Epoch: [2/10], Step: [7000/8000], Loss: 0.5493, Train_Acc: 0.5974
Epoch: [2/